In [1]:
%load_ext autoreload
%autoreload 2

import gymnasium as gym


from src.actor_critic import ActorCritic
from src.nets.mlp import Mlp
from src.rl_module import RlModule
import lightning as pl

In [2]:
# Initialise the environment
env = gym.make("LunarLander-v3")

# make the actor critic model
d = env.observation_space.shape[0]
n_a = env.action_space.n
hidden_dims = 64

actor_config = Mlp.config_cls(
    input_dims=d, output_dims=n_a, hidden_dims=hidden_dims, n_layers=1, activation="silu"
)
critic_config = Mlp.config_cls(
    input_dims=d, output_dims=1, hidden_dims=hidden_dims, n_layers=1, activation="silu"
)
ac_config = ActorCritic.config_cls(
    clip_epsilon=0.2, value_coeff=0.5, entropy_coeff=0.01
)

model = ActorCritic(ac_config, actor_net=Mlp(actor_config), critic=Mlp(critic_config))


config = RlModule.config_class()
trainer = pl.Trainer(max_epochs=25,)
module = RlModule(actor_critic=model, env=env, config=config)

trainer.fit(module)
# module

AttributeError: 'ActorCriticConfig' object has no attribute 'input_dims'

In [ ]:
# Initialise the environment
env = gym.make("LunarLander-v3", render_mode="human")

# Reset the environment to generate the first observation
observation, info = env.reset(seed=42)
for _ in range(5000):
    # this is where you would insert your policy
    action = module.predict(observation)

    # step (transition) through the environment with the action
    # receiving the next observation, reward and if the episode has terminated or truncated
    observation, reward, terminated, truncated, info = env.step(action)

    # If the episode has ended then we can reset to start a new episode
    if terminated or truncated:
        observation, info = env.reset()

env.close()

In [ ]:
actor